In [ ]:
h2 = 'https://www.metacritic.com/browse/games/release-date/available/pc/userscore'
i=0

driver=webdriver.Chrome(PATH)
driver.get(h2)
html = driver.find_element_by_tag_name('html')
html_final=html.get_attribute('innerHTML')

#metacritic crawling - getting list of urls:
soup = load_soup_object(html_final)


#getting pages links:
pages_links = soup.findAll('a' , attrs={'class':'page_num'})
pages_links_list = [str(i) for i in pages_links]

final_pages_list = list()
pages_link = pages_links_list[1][26:81]
for x in range (0,125):
    final_pages_list.append("https://www.metacritic.com"+pages_link+str(x))

cleaner_metaC_games = list()
cleaner_metaC_reviews = list()
cleaner_metaC_metascore = list()
metaC_rating_list = list()

#getting titles and ratings:
for x in range(0,metacritic_pages):
    # driver=webdriver.Chrome(PATH)
    driver.get('chrome://settings/clearBrowserData')
    driver.find_element_by_xpath('//settings-ui').send_keys(Keys.ENTER)
    driver.get(final_pages_list[x])
    html = driver.find_element_by_tag_name('html')
    html_final=html.get_attribute('innerHTML')
    soup = load_soup_object(html_final)

    metaC_game_title = soup.findAll('a' , attrs={'class':'title'})
    metaC_users_rating = soup.findAll('div' , attrs={'class':'clamp-score-wrap'})
    metaC_rating = soup.select('div.metascore_w.large.game')

    #cleaning titles:
    str_metaC_game_list = [str(i) for i in metaC_game_title]
    str_metaC_users_rating = [str(i) for i in metaC_users_rating]
    str_metaC_rating = [str(i) for i in metaC_rating]


    i=0
    for g in metaC_game_title:
        clean_metaC_game = bs(str_metaC_game_list[i] , "lxml").text
        cleaner_metaC_games.append(clean_metaC_game.strip())
        i+=1

    i=0
    for g in metaC_users_rating:
        clean_metaC_game = bs(str_metaC_users_rating[i] , "lxml").text
        cleaner_metaC_reviews.append(clean_metaC_game.strip())
        i+=1

    for p,w in enumerate(str_metaC_rating):
        result = str_metaC_rating[p].find('user')
        if (result > 0):
            continue
        else:
            metaC_rating_list.append(str_metaC_rating[p])

    
#creating the new df:
meta_df=pd.DataFrame({'GameTitles':cleaner_metaC_games , 'metaUsersScore':cleaner_metaC_reviews , 'meta_metaScore': metaC_rating_list})
meta_df.index+=1
meta_df['metaUsersScore'] = meta_df['metaUsersScore'].astype(float)

#clean meta_metaScore:
div_addons = ['<div class="metascore_w large game ','positive">','</div>','tbd">','mixed">','negative">']

for addon in div_addons:
    meta_df['meta_metaScore'] = meta_df['meta_metaScore'].str.replace(addon, '')

new_list = list ()
for index, w in enumerate(meta_df['meta_metaScore']):
    meta_df['meta_metaScore'][index+1].strip()
    word =  meta_df['meta_metaScore'][index+1]
    result = word.find('tbd')
    if result >= 0:
        meta_df['meta_metaScore'][index+1] = 0

meta_df['meta_metaScore'] = meta_df['meta_metaScore'].astype(int)

# meta_df.to_csv('rowMETADF.csv')


In [ ]:
meta_df = pd.read_csv('rowMETADF.csv')
meta_df = meta_df.drop(columns=['Unnamed: 0'])
meta_df.index+=1

df = pd.read_csv('cleanDF.csv')
df = df.drop(columns=['Unnamed: 0'])
df.index+=1

#Merging dataframes: 
meta_df['GameTitles'] = meta_df['GameTitles'].str.upper()
df_copy = df.copy(deep=True)
df_copy['GameTitles'] = df_copy['GameTitles'].str.upper()
df_copy = df_copy.merge(meta_df , how='left' , on='GameTitles')
df_copy.index+=1
for index, w in enumerate(df_copy['meta_metaScore']):
    if math.isnan(df_copy['meta_metaScore'][index+1]):
        df_copy['meta_metaScore'][index+1] = 0 

df_copy['meta_metaScore'] = df_copy['meta_metaScore'].astype(int)

for x, w in enumerate(df_copy['Metascore']):
    if(df_copy['meta_metaScore'][x+1]!= 0 and df_copy['Metascore'][x+1] == 'Noe') or ((df_copy['meta_metaScore'][x+1]!= 0 and df_copy['Metascore'][x+1] == 'None')):
        df_copy['Metascore'][x+1] = df_copy['meta_metaScore'][x+1]


In [ ]:

#creating a df copy and removing NaN:
df_copy = df_copy[df_copy['metaUsersScore'].notna()]
df_copy = df_copy[df_copy['Metascore'].notna()]
df_copy = df_copy[df_copy.Metascore != 'None']
df_copy = df_copy[df_copy.Metascore != 'Noe']
df_copy = df_copy[df_copy.Metascore != '']
df_copy = df_copy[df_copy['Storage(GB)'] != 0]
df_copy = df_copy[df_copy['ReleaseYear'] >= 1900]
df_copy = df_copy[df_copy['Storage(GB)'] <= 1000]
df_copy = df_copy[df_copy['SteamPositivePrecent'] > 65]
df_copy = df_copy.drop_duplicates(subset='GameTitles')
df_copy = df_copy.reset_index()
df_copy = df_copy.drop(columns=['index','meta_metaScore'])
df_copy.index+=1

df_copy['Metascore'] = df_copy['Metascore'].astype(int)
df_copy['Genre'] = df_copy['Genre'].astype(str)

for index,w in enumerate(df_copy['Genre']):
    df_copy['Genre'][index+1]=df_copy['Genre'][index+1].strip()

df_copy.to_csv('mergedDF.csv')

#importimg the merged df:

df_copy = pd.read_csv('mergedDF.csv')
df_copy = df_copy.drop(columns=['Unnamed: 0'])
df_copy.index+=1

#creating the num of labels column:

df_copy['numOfLabels'] = 0

count =0
for x,v in enumerate(df_copy['SinglePlayer']):
    count = df_copy['SinglePlayer'][x+1] + df_copy['onlineCo-op'][x+1] + df_copy['lanCo-op'][x+1] + df_copy['onlinePVP'][x+1] + df_copy['FullControllerSupport'] + df_copy['partialControllerSupport'][x+1] + df_copy['inAppPurchases'][x+1] + df_copy['MMO'][x+1] + df_copy['sharedSplitScreenPVP'][x+1]
    df_copy['numOfLabels'][x+1] = count.values[0]
    count = 0
